# **Preprocesamiento de imágenes**

El presente notebook tiene como objetivo el preprocesar las imágenes de ultrasonido antes de incorporarlas en el entrenamiento y evaluación de la inteligencia artificial.

## Resumen

Como ya se comentó en el archivo README.md, las imágenes de ultrasonido son en su totalidad capturas de pantallas del software donde se visualizan. Generalmente estas capturas de pantalla las generan los radiólogos para poder hacer apuntes o marcas que ayuden al diagnóstico por parte del médico. Se evaluaron diferentes aproximaciones, como el uso crudo de las imágenes o el recorte automatizado de estas.
Para la primera aproximación si bien se podría hacer un uso completo de todas las imágenes, la información del software que se cuela en la captura de pantalla podría ser irrelevante en el entrenamiento y de momento no se tiene claro que tanto lo podría alterar, por lo cual momentáneamente se descarta, aunque con base en los resultados futuros de entrenamiento quizá se podría retomar.
Para la segunda aproximación, a través de OpenCV (cv2) se detectaron los bordes de la imagen para generar el recorte. A veces se logró un buen resultado, pero en otras ocasiones fueron no fueron los deseados y no mejoraron por más que se ajustaran los parámetros. Después de un leve análisis se logró determinar que se podrían filtrar las imágenes funcionales a partir de su peso, siendo 350 kb el límite para considerarla funcional.

Con el procedimiento anterior se pasó de 2097 imágenes a poco más de 600, pero estas últimas nuevamente fueron filtradas, ya esta vez manualmente para asegurar su funcionalidad. A la par se revisaron parte de los nombres manualmente puesto que la clasificación que muchas imágenes tenían en el nombre estaba incorrecta.


# Metodología

## Recorte de imágenes

Son 2097 imágenes, lo que indica que el recorte manual, aunque posible, es una tarea muy demandante.

A partir de reconocimientos de bordes se intentó recortar cada una de las imágenes. Es importante destacar que antes de antes de esta metodología se intentaron otras haciendo uso de algoritmos como Canny o librerías como scikit-image, pero fue con OpenCV que se obtuvieron los mejores resultados.

In [ ]:
# Importamos librerías necesarias
import cv2
import numpy as np
from skimage import io, filters, measure
import os


def cortar_imagen(image_path, output_path):
    """
    Permite recortar una imágen de ultrasónido dentro de una captura
    de pantalla según los bordes de esta

    Args:
        image_path (str): ruta de la captura de pantalla
        output_path (str): ruta de destino de la imágen recortada
    """

    # Leer la imagen del camino especificado decartando el canal alfa
    imagen = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)[:,:,:3]

    # Convertir la imagen a escala de grises
    imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

    # Umbralizar la imagen para obtener una imagen binaria
    _, imagen_binaria = cv2.threshold(imagen_gris, 0, 255, cv2.THRESH_BINARY)

    # Encontrar contornos en la imagen binaria
    contornos, _ = cv2.findContours(imagen_binaria, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Encontrar el contorno más grande, basado en el área
    contorno_mas_grande = max(contornos, key=cv2.contourArea)

    # Obtener el rectángulo delimitador del contorno más grande
    x, y, ancho, alto = cv2.boundingRect(contorno_mas_grande)

    # Recortar la imagen al rectángulo delimitador
    imagen_recortada = imagen[y:y+alto, x:x+ancho]

    # Guardar la imagen recortada en el camino de salida especificado
    io.imsave(output_path, imagen_recortada)

# Ruta de las carpetas de las imágenes originales y destino de los
# recortes. Es importante que las carpetas contengan solo las imágenes
# que se van a procesar
input_folder = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/Dataset original/US_Pictures'
output_folder = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/images_test_4'

# Procesar cada imagen en la carpeta
for filename in os.listdir(input_folder):
    try:
        # Verificar si el archivo es una imagen basado en su extensión
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif')):
            # Construir la ruta completa de la imagen de entrada y de salida
            input_path = os.path.join(input_folder, filename)
            base_name = os.path.splitext(filename)[0]
            output_path = os.path.join(output_folder, base_name)
            # Llamar a la función para cortar y guardar la imagen
            cortar_imagen(input_path, output_path)
    except Exception as e:
        # Si hay un error, imprimirlo y continuar con la siguiente imagen
        print(f"No se pudo procesar el archivo: {filename}: {e}")

## Filtrado
Si bien se recortaron todas las imágenes, muchas de estas no cumplieron con los resultados esperados, llegando a ser cuadros negros o demasiados pequeños para llevar a cabo un análisis decente.

Se analizaron brevemente los resultados y se llegó a la conclusión de que generalmente si el peso de la imagen estaba por debajo de los 350 Kb significaba que esta NO serviría para el entrenamiento de la IA.

Antes de establecer el límite de 350 kb, se hizo un conteo proponiendo varios límites de peso entre el rango de 250 - 500 kb con el fin de no descartar excesivamente imágenes o quedar con muchas de poca calidad.


### Conteo

In [ ]:
import os

# Ruta a la carpeta donde están los archivos
folder_path = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/images_test_4'

# Tamaño en kilobytes
size_threshold = 350 * 1024  # 300 KB en bytes

# Contador para los archivos mayores a 300 KB
count = 0

# Listar todos los archivos en la carpeta
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    # Asegurarse de que es un archivo y no una carpeta/subcarpeta
    if os.path.isfile(file_path):
        # Obtener el tamaño del archivo
        file_size = os.path.getsize(file_path)
        # Comparar con el umbral
        if file_size > size_threshold:
            count += 1

print(f"Hay {count} archivos mayores a 300 KB.")

Hay 694 archivos mayores a 300 KB.


### Selección
Sabemos que hay al menos 694 imágenes que son más pesadas de 350 kb. Estas se copian en una nueva carpeta, se descargan y se revisan manualmente. La razón por la cual se optó por esta metodología fue por el tiempo del que se dispone, pues si bien parece una ardua labor, no se encontró ninguna otra forma en la cual extraer las imágenes y verificar automáticamente que estas estuvieran correctamente recortadas. Los autores reconocen que lo anterior no implica que sea imposible desarrollar una herramienta para tal labor o que esta no exista, pero no pudieron encontrarla e implementarla en los tiempos del presente trabajo.

Finalmente, si bien se tiene la copia de las imágenes en drive, es mucho más rápido procesarlas localmente, de allí el motivo de su descarga.

In [ ]:
import os
import shutil
import cv2
import numpy as np
from skimage import io, filters, measure
import os

# Ruta a la carpeta donde están los archivos originales
source_folder = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/images_test_4'

# Ruta a la carpeta donde quieres copiar los archivos
destination_folder = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/images_test_5'

# Tamaño en kilobytes
size_threshold = 350 * 1024  # 300 KB en bytes

# Contador para los archivos mayores a 300 KB
count = 0

# Listar todos los archivos en la carpeta
for filename in os.listdir(source_folder):
    source_path = os.path.join(source_folder, filename)
    # Asegurarse de que es un archivo y no una carpeta/subcarpeta
    if os.path.isfile(source_path):
        # Obtener el tamaño del archivo
        file_size = os.path.getsize(source_path)
        # Comparar con el umbral
        if file_size > size_threshold:
            # Copiar el archivo a la carpeta de destino
            destination_path = os.path.join(destination_folder, filename)
            shutil.copy2(source_path, destination_path)
            count += 1

print(f"Se copiaron {count} archivos mayores a 350 KB a {destination_folder}.")


Se copiaron 694 archivos mayores a 350 KB a /content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/images_test_5.


## Procesamiento

De aquí en adelante lo que procura es generar una base de datos con las imágenes la cual pueda ser usada para el entrenamiento del modelo a desarrollar.

Este preprocesamiento de imágenes debe ir orientado al modelo que se piensa desarrollar y para ello es importante recordar que la propuesta de este trabajo es el uso de redes neuronales convolucionales,, pero cuando se crea una red neuronal nueva todos sus parámetros inician de manera totalmente aleatoria y mediante el entrenamiento se ajustan gradualmente, el problema radica en que en este caso se cuenta únicamente con cientos de datos (580) lo que dificulta este proceso y aumentar el riesgo de caer en sobreajuste.

Por lo anterior se buscará hacer un proceso de "transferencia de aprendizaje" es decir, tomar un modelo que sea bueno reconociendo imágenes de cualquier tipo y entrenarlo sobre nuestro set. Se pretende usar [MobileNetV2](https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/5) donde se descartarían todas las salidas existentes del modelo (1001 clasificaciones), es decir, la útima capa, para remplazarla por otra con 3 clasificaciones ('sin apendice', 'apendicitis', 'no_apendicitis') y se entrenarían solo las últimas capas del modelo (sin tocar las otras), posteriormente se podrían entrenar todas las capas, pero con una tasa de aprendizaje muy baja. En este trabajo no se profundizará más sobre esa metodología, pero se deseaba que el lector comprendiese el camino a tomar.

### Ordenar imágenes.

Para la generación del modelo, se deben tener organizadas las imágenes y si bien algunas ya se organizaron manualmente, no fueron todas las categorias.

Se tomará la info del dataset estructurado que venía con las imágenes para saber si los grupos están bien clasificados, para ello se usarán los número de identificación US

#### Datos estructurados

In [ ]:
import pandas as pd

# Ruta del archivo
file_path = 'https://raw.githubusercontent.com/Dezarti/datos_monografia/main/structured_data/app_data.xlsx'

# Cargar la primera hoja del archivo Excel en un DataFrame
df = pd.read_excel(file_path, sheet_name=0)

# Mostrar las primeras filas del DataFrame
#df.head()

# Columnas seleccionadas
seleccion = ['Appendix_on_US', 'US_Number', 'Diagnosis']

# Especificar las filas (todas) y columnas (seleccion) que se copiaran
df_filtered = df.loc[:, seleccion]

df_filtered # Dataframe

,Appendix_on_US,US_Number,Diagnosis
0,yes,882.0,appendicitis
1,no,883.0,no appendicitis
2,no,884.0,no appendicitis
3,no,886.0,no appendicitis
4,yes,887.0,appendicitis
...,...,...,...
777,yes,126.0,appendicitis
778,no,NaN,appendicitis
779,no,127.0,appendicitis
780,yes,128.0,appendicitis


#### Revisión de valores únicos

Será útil para entender el dataset y no cometer errores al filtrar más adelante.

#### Mover imágenes de carpeta

Se extraen los numeros de las imágenes, se buscan en el dataset df_filtered, de allí se elige a qué carpeta debe ir con base en la información que se encuentre.

In [ ]:
import os # Para navegar por las carpetas de colab
import re # Para trabajar con expresiones regulares
import shutil # Para cambiar los archivos de ubicación
import traceback # Para resolver los problemas que se tienen mienstras se escribe el código

# DataFrame para llevar las cuentas de los archivos movidos.
df_results = pd.DataFrame({
    'us_number':[],
    'apendicitis':[],
    'no_apendicitis':[],
    'no_apendice':[],
    'no_class':[],
    'no_id':[]
})

# Ruta a la carpeta donde están las imágenes sin ordenar
folder_path = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/datasets/dataset_filtrado/app_visible'

# Rutas a las carpetas 'Mayor' y 'Menor'
apendicitis = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/datasets/dataset_ordenado/apendicitis'
no_apendicitis = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/datasets/dataset_ordenado/no_apendicitis'
no_apendice = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/datasets/dataset_ordenado/no_apendice'
no_classification = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/datasets/dataset_ordenado/no_classification'
no_id = '/content/drive/MyDrive/Materias/Monografía/Tercera entrega - 03 Noviembre/datasets/dataset_ordenado/no_ID'

# Función para extraer el número del nombre del archivo con ayuda
# de las funciones regulares
def extract_number(filename):
    match = re.match(r"(\d+)", filename)
    return int(match.group(1)) if match else None

# Procesar cada archivo en la carpeta
count = 0 # Para poder almacenar los resultados en df_results
for filename in os.listdir(folder_path):
    # Revisión que el archivo si corresponda a una imagen
    if filename.lower().endswith(('.png', '.bmp', '.jpeg', '.jpg', '.tif')):
        number = extract_number(filename) # número entero del archivo

        # Se busca en la base de datos filtrada si el número existe y se guardan
        # todos los valores de su fila
        fila_correspondiente = df_filtered.loc[df_filtered['US_Number'] == number]

        try:
            # Verifica que el número y la lista si existan
            if number is not None and fila_correspondiente.empty != True:
                df_results.loc[count, 'us_number'] = number

                # Revisa si el apendice aparece en la imagen
                if fila_correspondiente['Appendix_on_US'].iloc[0] == 'yes':

                    # Revisa si el diagnóstico es appendicitis o sin esta
                    if fila_correspondiente['Diagnosis'].iloc[0] == 'appendicitis':
                        target_folder = apendicitis
                        df_results.loc[count, 'apendicitis'] = 1
                        df_results.loc[count, ['no_apendicitis',
                                               'no_apendice',
                                               'no_id']] = 0

                    elif fila_correspondiente['Diagnosis'].iloc[0] == 'no_appendicitis':
                        target_folder = no_apendicitis
                        df_results.loc[count, 'no_apendicitis'] = 1
                        df_results.loc[count, ['apendicitis',
                                               'no_apendice',
                                               'no_id']] = 0

                    else:
                        target_folder = no_classification
                        df_results.loc[count, 'no_apendice'] = 1
                        df_results.loc[count, ['no_apendicitis',
                                               'apendicitis',
                                               'no_id']] = 0

                else:
                    target_folder = no_apendice
                    df_results.loc[count, 'no_apendice'] = 1
                    df_results.loc[count, ['no_apendicitis',
                                           'apendicitis',
                                           'no_id']] = 0

            # Imágenes cuyos pacientes NO existen en la base de datos
            else:
                target_folder = no_id
                df_results.loc[count, 'no_id'] = 1
                df_results.loc[count, ['no_apendicitis',
                                       'apendicitis',
                                       'no_apendice']] = 0

            # Determinar la carpeta de destino y mover el archivo
            #shutil.move(os.path.join(folder_path, filename),
                            #os.path.join(target_folder, filename))

        except Exception as e:
            # Si hay un error, imprimirlo y continuar con la siguiente imagen
            print(f"No se pudo procesar el archivo: {filename}: {e}")
            traceback.print_exc() # Imprime el tipo de error

    count += 1

Imágenes que podría usar para no_apendicitis

In [ ]:
# Imágenes donde se ve el apéndice y NO tiene apendicitis

df_filtered2 = df_filtered[df_filtered['Appendix_on_US'] == 'yes']

df_filtered2 = df_filtered2[df_filtered2['Diagnosis'] == 'no appendicitis']

df_filtered2.count()

Appendix_on_US    126
US_Number         125
Diagnosis         126
dtype: int64

In [ ]:
# Imágenes donde se ve el apéndice y tiene apendicitis

df_filtered2 = df_filtered[df_filtered['Appendix_on_US'] == 'yes']

df_filtered2 = df_filtered2[df_filtered2['Diagnosis'] == 'appendicitis']

df_filtered2.count()

Appendix_on_US    378
US_Number         368
Diagnosis         378
dtype: int64

In [ ]:
# Imágenes donde se ve el apéndice y tiene apendicitis

df_filtered2 = df_filtered[df_filtered['Appendix_on_US'] == 'no']

df_filtered2.count()

Appendix_on_US    273
US_Number         263
Diagnosis         273
dtype: int64

# Referencias

-
-